<a href="https://colab.research.google.com/github/callor/Callor-DeepLearning-2022/blob/master/%ED%95%98%EC%9D%B4%ED%8D%BC_%ED%8C%8C%EB%9D%BC%EB%A9%94%ED%84%B0_%EC%B5%9C%EC%A0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이렇게 하려면 케라스 모델을 사이킷런 추정기처럼 보이게 바꿔야합니다.<br>
먼저 일련의 하이퍼파라미터로 케라스 모델을 만들고 컴파일하는 함수를 만듭니다.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
housing = fetch_california_housing()

In [5]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

X_new = X_test[:3]

In [6]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

build_model() 함수를 사용해 KerasRegressor 클래스 객체를 만듭니다.

In [7]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

<ipython-input-7-651c14c6d32f>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


이 객체는 build_model() 함수로 만들어진 케라스 모델을 감싸는 간단한 래퍼입니다.<br>
이 객체를 만들 때 어떤 하이퍼파라미터도 정의하지 않았으므로 build_model()에 정의된 기본 파라미터를 사용할 것입니다.

이제 일반적 사이킷런 회귀 추정기처럼 이 객체를 사용할 수 있습니다.

In [8]:
keras_reg.fit(X_train, y_train, epochs=100,
             validation_data=(X_valid, y_valid),
             callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

Epoch 1/100
363/363 [==============================] - 1s 2ms/step - loss: 1.0896 - val_loss: 20.7721
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.7606 - val_loss: 5.0266
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5456 - val_loss: 0.5490
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4732 - val_loss: 0.4529
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4503 - val_loss: 0.4188
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4338 - val_loss: 0.4129
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4241 - val_loss: 0.4004
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4168 - val_loss: 0.3944
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4108 - val_loss: 0.3961
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4060 - val_l

fit() 메소드에 지정한 모든 매개변수는 케라스 모델로 전달됩니다.<br>
사이킷런은 손실이 아니라 점수를 계산하기 때문에 출력 점수는 음수의 MSE입니다.

모델 하나를 훈련하고 평가하려는 게 아니라 수백 개의 모델을 훈련하고<br>
검증 세트에서 최상의 성능을 보이는 모델을 선택해야 합니다.

하이퍼파라미터가 많으므로 그리드탐색보다 랜덤 탐색을 사용하는 게 좋습니다.<br>
은닉층 개수, 뉴런 개수, 학습률을 사용해 하이퍼파라미터 탐색을 해보겠습니다.

In [9]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_distribs = {
    'n_hidden': [0, 1, 2, 3],
    'n_neurons': np.arange(1, 100),
    'learning_rate': reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                 validation_data=(X_valid, y_valid),
                 callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/100
242/242 [==============================] - 2s 7ms/step - loss: 3.3071 - val_loss: 1.3735
Epoch 2/100
242/242 [==============================] - 1s 5ms/step - loss: 0.9782 - val_loss: 0.6881
Epoch 3/100
242/242 [==============================] - 1s 5ms/step - loss: 0.6186 - val_loss: 0.5887
Epoch 4/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5504 - val_loss: 0.5466
Epoch 5/100
242/242 [==============================] - 1s 4ms/step - loss: 0.5363 - val_loss: 0.5360
Epoch 6/100
242/242 [==============================] - 2s 7ms/step - loss: 0.5307 - val_loss: 0.7261
Epoch 7/100
242/242 [==============================] - 1s 4ms/step - loss: 0.5322 - val_loss: 0.6728
Epoch 8/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5311 - val_loss: 0.6939
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5326 - val_loss: 0.5351
Epoch 10/100
242/242 [========

RandomizedSearchCV는 k-겹 교차 검증을 사용하기 때문에 X_valid와 y_valid를 사용하지 않습니다.<br>
이 데이터는 조기 종료 시에만 사용됩니다.

다음은 하이퍼파라미터 최적화에 사용할 수 있는 다른 파이썬 라이브러리들입니다.

- Hyperopt
- Hyperas, kopt, Talos
- Keras Tuner
- Scikit-Optimize(skopt)
- Spearmint
- Hyperband
- Sklearn-Deap